In [52]:
import pandas as pd
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

In [87]:
og_data = pd.read_excel(rf'Files/TXS que el collector no ve NOV23 al final del mes.xlsx')
og_data = og_data.rename(columns={'Id_x':'Id'})
og_data['Date'] = og_data.CreatedAt.dt.date

In [88]:
og_data = og_data.astype({'TransactionId':'str', 'SenderPhoneNumber':'str' })

# MSFT GUIDE

# Feature Engineering

In [89]:
og_data = og_data[og_data.Status=='COMPLETED']
og_data

,CreatedAt,Id,TransactionId,Client,VendorCode,Status,CollectMethod,TargetCountry,TargetCurrency,NetAmountUSD,...,senderCountry,receiverFirstName,receiverLastName,receiverDocument,receiverDocumentType,receiverPhoneNumber,receiverCountry,receiverBankAccountNumber,receiverBankAccountType,Date
0,2023-11-22 17:31:27.991,90BDB5C9-AD84-4C1A-85D5-5C106BA6FA2A,1277528,SANTANDERCHL,TRANSFERZERO,COMPLETED,BANK_ACCOUNT,ESP,EUR,5441.022042,...,CHL,Mirko,Zlatar sepulveda,X3660543R,5,+34651310338,ESP,ES5000817440190002109412,1.0,2023-11-22
1,2023-11-30 21:04:50.916,81465787-552A-49CE-8DA2-F23D4E09A796,1302804,SANTANDERCHL,TRANSFERZERO,COMPLETED,BANK_ACCOUNT,ESP,EUR,2721.651280,...,CHL,Ana Isabel,Munoz Moreno,21475223P,5,+56979636853,ESP,ES9300494558962790001655,1.0,2023-11-30
2,2023-11-07 15:24:58.947,4E8FC322-555C-4CBF-AF4A-BA2180FA75B0,1226955,SANTANDERCHL,COBRU,COMPLETED,BANK_ACCOUNT,COL,COP,2548.889692,...,CHL,Juan David,Alvarez Palau,75102633,5,+573017877862,COL,1008494149,0.0,2023-11-07
3,2023-11-02 19:59:14.868,0C5381FA-23F5-4016-BDF5-298D6A64C773,1210171,SANTANDERCHL,TRANSFERZERO,COMPLETED,BANK_ACCOUNT,ESP,EUR,2230.651488,...,CHL,Bernardita Pilar,Castro Garay,Y9154869D,5,+34633810788,ESP,ES3800810646350001642765,1.0,2023-11-02
4,2023-11-30 19:48:12.852,A767BF8B-03F6-4825-9A7A-E3E65A05F4FA,1302462,SANTANDERCHL,TRANSFERZERO,COMPLETED,BANK_ACCOUNT,ESP,EUR,2178.305170,...,CHL,VALENTINA,VASQUEZ,04321243A,5,+347221468220,ESP,ES1600490542722213192466,1.0,2023-11-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,2023-11-17 17:50:35.043,8202FC47-5CC1-48F0-AD2E-42C3D481E818,1262869,BANCO_ESTADO,ITALCAMBIO,COMPLETED,BANK_ACCOUNT,VEN,VEF,1.770000,...,CHL,Solanyela,Gonzalez,V17081027,5,+584124525698,VEN,01020467410000403021,NaN,2023-11-17
577,2023-11-18 17:21:23.449,830380AA-847F-434A-B3C7-CEDD18EFC494,1265920,BANCO_ESTADO,ITALCAMBIO,COMPLETED,BANK_ACCOUNT,VEN,VEF,1.430000,...,CHL,Maury Alejandra,Pena,V18349723,5,+584269013912,VEN,01910549191100037452,NaN,2023-11-18
578,2023-11-27 15:04:02.957,D20C555C-7414-4369-9048-6F162191295B,1290947,REMITEE,BANCOINDUSTRIAL,COMPLETED,BANK_ACCOUNT,ARG,ARS,1.392705,...,ARG,Gian,Ghiano,23380109409,7,+5491151754767,ARG,0070175030004051288535,NaN,2023-11-27
579,2023-11-30 19:41:06.440,BACA283D-C79E-4E28-9F88-DC285D6FD262,1302413,REMITEE,BANCOINDUSTRIAL,COMPLETED,BANK_ACCOUNT,ARG,ARS,1.387071,...,ARG,Test,TransaccionBancaria,23380109409,7,+5491151754767,ARG,0070175030004051288535,NaN,2023-11-30


In [90]:
og_data = og_data[['CreatedAt', 'Date', 'Id', 'TransactionId', 'Client', 'VendorCode', 'TargetCountry', 'TargetCurrency', 'NetAmountUSD',
       'SourceCountry', 'SenderFirstName',
       'SenderLastName', 'SenderDocument', 'SenderPhoneNumber',
       'senderCountry', 'receiverFirstName', 'receiverLastName',
       'receiverDocument', 'receiverDocumentType', 'receiverPhoneNumber',
       'receiverCountry', 'receiverBankAccountNumber',
       'receiverBankAccountType']]

In [91]:
def get_zscore(value, mean, std):
    # calculate z-score or number of standard deviations from mean
    if (
        std == 0
        or std is None
        or str(std).lower() in ["nan", "none", "null"]
        or mean is None
    ):
        if value == 0.0:
            return 0.0
        elif value != 0:
            return value #np.log10(value + 1)
    ans = (value - mean) / std
    # only interested in increases
    #ans = max(0.0, ans)
    # take log to dampen numbers
    #ans = np.log10(ans + 1)
    return float(ans)

In [92]:
data = og_data.copy()


zscore_columns = [
    "NetAmountUSD"
]

#usamos documento del enviador y phone number del recibidor porque son los que mas hay
ind = ['Date', 'Client', 'VendorCode', 'SourceCountry', 'TargetCountry', 'TargetCurrency', 
    'SenderDocument', 'receiverPhoneNumber', 'receiverBankAccountNumber']

means = [x + "_" + y + "_mean" for x in zscore_columns for y in ind]
stds = [x + "_" + y + "_std" for x in zscore_columns for y in ind]
zscores = [x + "_" + y + "_zscore" for x in zscore_columns for y in ind]

zscore = data[zscore_columns + ind]
zscore = zscore.fillna(0)


for metric, index in zip(means,ind):
    zscore[metric] = zscore.groupby([index])[zscore_columns].transform("mean")

for metric, index in zip(stds,ind):
    zscore[metric] = zscore.groupby([index])[zscore_columns].transform("std", ddof=1)


zscore = zscore.drop_duplicates(ind)
zscore = zscore[means + stds + ind]
data = data.merge(zscore, how="left", on=ind)


from itertools import product

for column,feature in product(zscore_columns, ind):
    data[f"{column}_{feature}_zscore"] = data.apply(
        lambda row: get_zscore(
            row[f"{column}"], row[f"{column}_{feature}_mean"], row[f"{column}_{feature}_std"]
        ),
        axis=1,
    )


### Counts by sender

In [93]:
sender_count_columns=['Id', 'Date', 'Client', 'VendorCode', 'TargetCountry', 'TargetCurrency',  'receiverPhoneNumber', 'receiverBankAccountNumber']
sender_count_ind = ['SenderDocument']

sender_counts = ["Sender_" + x + "_count" for x in sender_count_columns]

sender_count = data[sender_count_columns + sender_count_ind]
sender_count = sender_count.fillna(0)

for column  in sender_count_columns:
    #print(column)
    sender_count[f"Sender_{column}_count"] = sender_count.groupby('SenderDocument')[column].transform("nunique")

sender_count = sender_count.drop_duplicates(sender_count_ind)
sender_count = sender_count[sender_counts + sender_count_ind]

for att in sender_counts:
    if (sender_count[att].max()==1):
        sender_count.drop(columns={att}, inplace=True)


In [94]:
data = data.merge(sender_count, how="left", on=sender_count_ind)

### Counts by receiver

In [95]:
receiver_count_columns=['Id', 'Date', 'Client', 'VendorCode', 'SourceCountry', 'SenderDocument']
receiver_count_ind = ['receiverPhoneNumber']

receiver_counts = ["receiver_" + x + "_count" for x in receiver_count_columns]

receiver_count = data[receiver_count_columns + receiver_count_ind]
receiver_count = receiver_count.fillna(0)

for column  in receiver_count_columns:
    #print(column)
    receiver_count[f"receiver_{column}_count"] = receiver_count.groupby('receiverPhoneNumber')[column].transform("nunique")

receiver_count = receiver_count.drop_duplicates(receiver_count_ind)
receiver_count = receiver_count[receiver_counts + receiver_count_ind]

for att in receiver_counts:
    if (receiver_count[att].max()==1):
        receiver_count.drop(columns={att}, inplace=True)


In [96]:
data = data.merge(receiver_count, how="left", on=receiver_count_ind)

### One hot encoding of categorical variables

In [105]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
categorical_features = ['Client', 'VendorCode','SourceCountry','TargetCountry', 'TargetCurrency']
X=data[categorical_features]

enc.fit(X)
codes = enc.transform(X).toarray()
encoded_categorical_features = enc.get_feature_names_out(categorical_features)


In [117]:
data = pd.concat([data.drop(columns=categorical_features), pd.DataFrame(codes,columns=encoded_categorical_features).astype(int)], axis=1)
encoded_categorical_features

array(['Client_BANCO_ESTADO', 'Client_CHEK', 'Client_MONEYGRAM2',
       'Client_MOVII', 'Client_PAPAYA', 'Client_PAYSEND', 'Client_PMI',
       'Client_PREXCARD', 'Client_REMITEE', 'Client_REMITLY',
       'Client_SANTANDERCHL', 'Client_TENPO', 'Client_UNITELLER',
       'VendorCode_BANCOINDUSTRIAL', 'VendorCode_COBRU',
       'VendorCode_COINAG', 'VendorCode_DLOCAL', 'VendorCode_EASYPAGOS',
       'VendorCode_INTER', 'VendorCode_ITALCAMBIO',
       'VendorCode_LOCALPAYMENT', 'VendorCode_PONTUAL',
       'VendorCode_RADAR', 'VendorCode_RIPPLENET',
       'VendorCode_TRANSFERZERO', 'VendorCode_UNITELLER',
       'VendorCode_interbank', 'SourceCountry_ARE', 'SourceCountry_ARG',
       'SourceCountry_AUS', 'SourceCountry_AUT', 'SourceCountry_CAN',
       'SourceCountry_CHL', 'SourceCountry_COL', 'SourceCountry_DEU',
       'SourceCountry_DNK', 'SourceCountry_ECU', 'SourceCountry_ESP',
       'SourceCountry_EST', 'SourceCountry_GBR', 'SourceCountry_IRL',
       'SourceCountry_ITA', 'Sourc

In [ ]:
X

# Isolation Forest application

In [123]:
def apply_isolation_forest(df, n_estimators, contamination=0.01):
    """Applies Isolation Forest to a given dataset and returns the predicted anomalies."""
    clf = IsolationForest(
        n_estimators=n_estimators,
        max_samples="auto",
        contamination=contamination,
        max_features=6,
        bootstrap=False,
        n_jobs=-1,
        random_state=42,
        verbose=0,
    )
    clf.fit(df.values)
    pred = clf.predict(df.values)
    scores = clf.decision_function(df.values)
    return clf, pred, scores

# specify the metrics column names to be modelled
features = [
    'NetAmountUSD', 
       'NetAmountUSD_Date_mean',
       'NetAmountUSD_Client_mean', 'NetAmountUSD_VendorCode_mean',
       'NetAmountUSD_SourceCountry_mean', 'NetAmountUSD_TargetCountry_mean',
       'NetAmountUSD_TargetCurrency_mean', 'NetAmountUSD_SenderDocument_mean',
       'NetAmountUSD_receiverPhoneNumber_mean',
       'NetAmountUSD_receiverBankAccountNumber_mean', 'NetAmountUSD_Date_std',
       'NetAmountUSD_Client_std', 'NetAmountUSD_VendorCode_std',
       'NetAmountUSD_SourceCountry_std', 'NetAmountUSD_TargetCountry_std',
       'NetAmountUSD_TargetCurrency_std', 'NetAmountUSD_SenderDocument_std',
       'NetAmountUSD_receiverPhoneNumber_std',
       'NetAmountUSD_receiverBankAccountNumber_std',
       'NetAmountUSD_Date_zscore', 'NetAmountUSD_Client_zscore',
       'NetAmountUSD_VendorCode_zscore', 'NetAmountUSD_SourceCountry_zscore',
       'NetAmountUSD_TargetCountry_zscore',
       'NetAmountUSD_TargetCurrency_zscore',
       'NetAmountUSD_SenderDocument_zscore',
       'NetAmountUSD_receiverPhoneNumber_zscore',
       'NetAmountUSD_receiverBankAccountNumber_zscore', 'Sender_Id_count',
       'Sender_Date_count', 'Sender_VendorCode_count',
       'Sender_receiverPhoneNumber_count',
       'Sender_receiverBankAccountNumber_count', 'receiver_Id_count',
       'receiver_Date_count', 'receiver_VendorCode_count',
       'receiver_SenderDocument_count'
] + list(encoded_categorical_features)


In [124]:
X = data[features].copy()
X = X.fillna(0)

if X.shape[0] < 500:
    n_estimators = len(features) * 4 + X.shape[0] * 2
else:
    n_estimators = 100

# n_estimators = 100
print("Number of trees", n_estimators)

clf, pred, scores = apply_isolation_forest(X, n_estimators, contamination=0.01)
data["anomaly"] = pred
data["score"] = scores
# excluding users who do not have any successful logon history.


Number of trees 100


In [127]:
outliers = data.loc[data["anomaly"] == -1]
outlier_index = list(outliers.index)
print(f"Outliers at indexes: {outlier_index}")
# Find the number of anomalies and normal points here points classified -1 are anomalous
print(data["anomaly"].value_counts())

Outliers at indexes: [0, 2, 3, 4, 9, 12]
 1    549
-1      6
Name: anomaly, dtype: int64


In [128]:
print("Number of outliers:", outliers.shape[0])
print("Top anomalies by score (top most has highest anomaly score and so on)")
outliers = outliers.sort_values(by=["score"], ascending=False)
display(outliers.head())

Number of outliers: 6
Top anomalies by score (top most has highest anomaly score and so on)


,CreatedAt,Date,Id,TransactionId,NetAmountUSD,SenderFirstName,SenderLastName,SenderDocument,SenderPhoneNumber,senderCountry,...,TargetCurrency_BRL,TargetCurrency_CLP,TargetCurrency_COP,TargetCurrency_EUR,TargetCurrency_MXN,TargetCurrency_PEN,TargetCurrency_USD,TargetCurrency_VEF,anomaly,score
12,2023-11-06 23:09:04.561,2023-11-06,78408B13-F91A-4EB4-91CC-D7551C52E70A,1225028,1071.490714,ESTELA PAOLA,GARVIZU AGUILERA,Y09N87W4L,nan,DEU,...,0,0,0,0,0,0,0,0,-1,-0.001657
9,2023-11-02 15:27:04.683,2023-11-02,5923AF69-83A1-4FF4-AFC9-8B326CF91256,1209043,1492.560523,Uriel,Bertran,46702672E,nan,ESP,...,0,0,1,0,0,0,0,0,-1,-0.009308
4,2023-11-30 19:48:12.852,2023-11-30,A767BF8B-03F6-4825-9A7A-E3E65A05F4FA,1302462,2178.305170,WILLIAM LARRY,VASQUEZ ORTEGA,00109178497,56966098213.0,CHL,...,0,0,0,1,0,0,0,0,-1,-0.012021
3,2023-11-02 19:59:14.868,2023-11-02,0C5381FA-23F5-4016-BDF5-298D6A64C773,1210171,2230.651488,MARIA RAQUEL,MEJIAS SILVA,00052005191,56952214701.0,CHL,...,0,0,0,1,0,0,0,0,-1,-0.014998
2,2023-11-07 15:24:58.947,2023-11-07,4E8FC322-555C-4CBF-AF4A-BA2180FA75B0,1226955,2548.889692,BRAYHAN BORIS,BUENO CORTES,00244993249,56967886270.0,CHL,...,0,0,1,0,0,0,0,0,-1,-0.032611


# Data Viz

In [130]:
import plotly.express as px

data["anomaly_label"] = data["anomaly"].apply(
    lambda x: "outlier" if x == -1 else "inlier"
)
fig = px.histogram(data, x="score", color="anomaly_label")
fig.show()

In [141]:
data.fillna(0)

,CreatedAt,Date,Id,TransactionId,NetAmountUSD,SenderFirstName,SenderLastName,SenderDocument,SenderPhoneNumber,senderCountry,...,TargetCurrency_CLP,TargetCurrency_COP,TargetCurrency_EUR,TargetCurrency_MXN,TargetCurrency_PEN,TargetCurrency_USD,TargetCurrency_VEF,anomaly,score,anomaly_label
0,2023-11-22 17:31:27.991,2023-11-22,90BDB5C9-AD84-4C1A-85D5-5C106BA6FA2A,1277528,5441.022042,MIRKO,ZLATAR SEPULVEDA,00103233747,56998742667.0,CHL,...,0,0,1,0,0,0,0,-1,-0.076002,outlier
1,2023-11-30 21:04:50.916,2023-11-30,81465787-552A-49CE-8DA2-F23D4E09A796,1302804,2721.651280,RAFAEL,MARTINEZ NEIRA,00277829045,56993201618.0,CHL,...,0,0,1,0,0,0,0,1,0.002783,inlier
2,2023-11-07 15:24:58.947,2023-11-07,4E8FC322-555C-4CBF-AF4A-BA2180FA75B0,1226955,2548.889692,BRAYHAN BORIS,BUENO CORTES,00244993249,56967886270.0,CHL,...,0,1,0,0,0,0,0,-1,-0.032611,outlier
3,2023-11-02 19:59:14.868,2023-11-02,0C5381FA-23F5-4016-BDF5-298D6A64C773,1210171,2230.651488,MARIA RAQUEL,MEJIAS SILVA,00052005191,56952214701.0,CHL,...,0,0,1,0,0,0,0,-1,-0.014998,outlier
4,2023-11-30 19:48:12.852,2023-11-30,A767BF8B-03F6-4825-9A7A-E3E65A05F4FA,1302462,2178.305170,WILLIAM LARRY,VASQUEZ ORTEGA,00109178497,56966098213.0,CHL,...,0,0,1,0,0,0,0,-1,-0.012021,outlier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,2023-11-17 17:50:35.043,2023-11-17,8202FC47-5CC1-48F0-AD2E-42C3D481E818,1262869,1.770000,SOLANYELA,GONZALEZ -,274083433,nan,CHL,...,0,0,0,0,0,0,1,1,0.131168,inlier
551,2023-11-18 17:21:23.449,2023-11-18,830380AA-847F-434A-B3C7-CEDD18EFC494,1265920,1.430000,MAURY,PENA -,260520652,nan,CHL,...,0,0,0,0,0,0,1,1,0.128744,inlier
552,2023-11-27 15:04:02.957,2023-11-27,D20C555C-7414-4369-9048-6F162191295B,1290947,1.392705,DANA ANABEL,OVALLE,41.879.837,5491125883828.0,ARG,...,0,0,0,0,0,0,0,1,0.095757,inlier
553,2023-11-30 19:41:06.440,2023-11-30,BACA283D-C79E-4E28-9F88-DC285D6FD262,1302413,1.387071,DANA ANABEL,OVALLE,41.879.837,5491125883828.0,ARG,...,0,0,0,0,0,0,0,1,0.099517,inlier


In [137]:
from pathlib import Path
import os
import sys
import warnings
import shap
from IPython.display import display, HTML, Markdown, Javascript
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import IsolationForest

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [143]:
pca3 = PCA(n_components=3)  # Reduce to k=3 dimensions
scaler = StandardScaler()
# normalize the metrics
X = scaler.fit_transform(data.fillna(0)[features])
X_reduce = pca3.fit_transform(X)


In [151]:


# Add destination user to label
data["labels"] = np.where(
    data["anomaly"] == -1,
    data["TransactionId"].astype("str"),
    "non-anomalous",
)


In [152]:

total_var = pca3.explained_variance_ratio_.sum() * 100
fig = px.scatter_3d(
    X_reduce,
    x=0,
    y=1,
    z=2,
    color=data["labels"],
    title=f"Total Explained Variance: {total_var:.2f}%",
    labels={
        "0": "Principal Component 1",
        "1": "Principal Component 2",
        "2": "Principal Component 3",
    },
)
fig.show()